In [2]:
import os
import time
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import final_project.config as c
from final_project.models.Frame import Frame
from final_project.models.Matcher import Matcher, DEFAULT_MATCHER

In [21]:

def measure_inliers_within_frame(n: int, m: Matcher, verbose=False):
    assert 0 < n <= c.NUM_FRAMES
    start = time.time()

    match_count = np.zeros(n)
    feature_count = np.zeros(n)
    for i in range(n):
        img_l, img_r = c.read_images(i)
        kps_l, descs_l = m._detector.detectAndCompute(img_l, None)
        kps_r, descs_r = m._detector.detectAndCompute(img_r, None)
        all_matches = m.match_descriptors(descs_l, descs_r)
        features, descriptors = m._find_frame_inliers(kps_l, kps_r, descs_l, all_matches)
        match_count[i] = len(all_matches)
        feature_count[i] = len(features)
        if i%200==0 and verbose:
            elapsed = time.time() - start
            print(f"\tFinished {i} Frames in {elapsed:.2f} seconds")

    elapsed = time.time() - start
    return match_count, feature_count, elapsed


In [ ]:
N = c.NUM_FRAMES

# default values
m0 = Matcher(detector_type=c.DEFAULT_DETECTOR_NAME, matcher_type=c.DEFAULT_MATCHER_NAME, use_crosscheck=c.SHOULD_CROSS_CHECK, use_2nn=c.SHOULD_USE_2NN)
match_count0, feature_count0, elapsed0 = measure_inliers_within_frame(N, m0, verbose=True)
print(f"Finished m0 in {(elapsed0/60):.2f} minutes")

# expected to be the fastest
m1 = Matcher(detector_type="orb", matcher_type="bf", use_crosscheck=False, use_2nn=False)
match_count1, feature_count1, elapsed1 = measure_inliers_within_frame(N, m1, verbose=True)
print(f"Finished m1 in {(elapsed1/60):.2f} minutes")

# expected to be slow
m2 = Matcher(detector_type="sift", matcher_type="bf", use_crosscheck=False, use_2nn=True)
match_count2, feature_count2, elapsed2 = measure_inliers_within_frame(N, m2, verbose=True)
print(f"Finished m2 in {(elapsed2/60):.2f} minutes")

# expected to be slow
m3 = Matcher(detector_type="sift", matcher_type="bf", use_crosscheck=True, use_2nn=False)
match_count3, feature_count3, elapsed3 = measure_inliers_within_frame(N, m2, verbose=True)
print(f"Finished m3 in {(elapsed2/60):.2f} minutes")

# expected to be slow
m4 = Matcher(detector_type="sift", matcher_type="flann", use_crosscheck=False, use_2nn=False)
match_count4, feature_count4, elapsed4 = measure_inliers_within_frame(N, m2, verbose=True)
print(f"Finished m4 in {(elapsed2/60):.2f} minutes")

	Finished 0 Frames in 0.70 seconds
	Finished 200 Frames in 112.88 seconds
	Finished 400 Frames in 174.83 seconds
	Finished 600 Frames in 267.90 seconds
	Finished 800 Frames in 387.20 seconds
	Finished 1000 Frames in 549.47 seconds
	Finished 1200 Frames in 699.52 seconds
	Finished 1400 Frames in 783.75 seconds
	Finished 1600 Frames in 891.62 seconds
	Finished 1800 Frames in 1003.82 seconds
	Finished 2000 Frames in 1173.83 seconds
	Finished 2200 Frames in 1313.21 seconds
	Finished 2400 Frames in 1504.52 seconds
	Finished 2600 Frames in 1573.29 seconds
	Finished 2800 Frames in 1684.91 seconds
	Finished 3000 Frames in 1816.48 seconds
	Finished 3200 Frames in 1917.50 seconds
	Finished 3400 Frames in 2038.63 seconds
Finished m0 in 34.40 minutes
	Finished 0 Frames in 0.04 seconds
	Finished 200 Frames in 6.44 seconds
	Finished 400 Frames in 11.59 seconds
	Finished 600 Frames in 18.03 seconds
	Finished 800 Frames in 25.01 seconds
	Finished 1000 Frames in 31.95 seconds
	Finished 1200 Frames in 3

In [ ]:
frame_ids = [i+1 for i in range(c.NUM_FRAMES)]
colorblind_friendly = ["#762a83", "#de77ae", "gold", "cyan", "#1b7837"]

plt.clf()
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 4))
gs = axes[0, 1].get_gridspec()  # see tutorial: https://matplotlib.org/stable/gallery/subplots_axes_and_figures/gridspec_and_subplots.html
for ax in axes[:, 1]:
    ax.remove()
ax_big = fig.add_subplot(gs[:, 1])
# fig.tight_layout()

axes[0].set_title("Runtime")
axes[0].bar(x=[i for i in range(1,6)], height=[elapsed0, elapsed1, elapsed2, elapsed3, elapsed4],
            linewidth=0, colors=colorblind_friendly, tick_label=[[f"M{i}" for i in range(1, 6)]])

axes[0, 0].set_title("# Inliers")
axes[0, 0].plot(frame_ids, feature_count0, linewidth=1, c=colorblind_friendly[0], label="M1")
axes[0, 0].plot(frame_ids, feature_count1, linewidth=1, c=colorblind_friendly[1], label="M2")
axes[0, 0].plot(frame_ids, feature_count2, linewidth=1, c=colorblind_friendly[2], label="M3")
axes[0, 0].plot(frame_ids, feature_count3, linewidth=1, c=colorblind_friendly[3], label="M4")
axes[0, 0].plot(frame_ids, feature_count4, linewidth=1, c=colorblind_friendly[4], label="M5")
axes[0, 0].set_xlabel("Frame ID")

axes[1, 0].set_title("% Inliers")
axes[1, 0].plot(frame_ids, 100 * np.divide(feature_count0, match_count0), linewidth=1, c=colorblind_friendly[0], label="M1")
axes[1, 0].plot(frame_ids, 100 * np.divide(feature_count1, match_count1), linewidth=1, c=colorblind_friendly[1], label="M2")
axes[1, 0].plot(frame_ids, 100 * np.divide(feature_count2, match_count2), linewidth=1, c=colorblind_friendly[2], label="M3")
axes[1, 0].plot(frame_ids, 100 * np.divide(feature_count3, match_count3), linewidth=1, c=colorblind_friendly[3], label="M4")
axes[1, 0].plot(frame_ids, 100 * np.divide(feature_count4, match_count4), linewidth=1, c=colorblind_friendly[4], label="M5")
axes[1, 0].set_xlabel("Frame ID")

plt.show()

In [19]:
orb = cv2.ORB_create()
sift = cv2.SIFT_create()

def detect(detector, img_l, img_r):
    start = time.time()
    kps_l, descs_l = detector.detectAndCompute(img_l, None)
    kps_r, descs_r = detector.detectAndCompute(img_r, None)
    elapsed = time.time()
    return kps_l, descs_l, kps_r, descs_r, elapsed

orb_results = []
sift_results = []
for i in range(c.NUM_FRAMES):
    img_l, img_r = c.read_images(i)
    orb_results.append(detect(orb, img_l, img_r))
    sift
    orb_kps_l, orb_descs_l, orb_kps_r, orb_descs_r, orb_elapsed = detect(orb)
    sift_kps_l, sift_descs_l, sift_kps_r, sift_descs_r, sift_elapsed = detect(sift)